# Compare the simulation results with existing charging infrastructure in Sweden

In [23]:
%load_ext autoreload
%autoreload 2
%cd D:\synthetic-sweden
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
D:\synthetic-sweden


In [24]:
import pandas as pd
import geopandas as gpd
import dataworkers as dw
scenario = 'scenario_vg_car'

## Load today's infrastructure

In [25]:
df = pd.read_excel('dbs/ev/charging_sites.xlsx')
df.head()

,lm,id,nm,adr,pnr,rsp,rsb,rcd,cty,ctryCd,...,pw,n,cd,lm3,oprtr,cnm,phrs,npp,prte,ownr
0,2022-06-14T17-27-54,a6232963-fe8d-44c2-8a33-f1b1f97f2fc8,Pendelparkering E6 Kodemotet,Kodemotet pendelparkering,44260,Västra Götalands län,Kungälv,1482,Kode,SE,...,3.7,6.0,2014-07-07T12:20:48.5187849+02:00,2020-11-11T23:30:08.4066729+01:00,Kungälv energi,NaN,Dygnet runt,6,Gratis,Kungälvs kommun
1,2022-06-14T17-27-54,ea1d02aa-fa06-400f-b141-6d5f024f9b34,P-hus Olskroken,Lilla Olskroksgatan 1,416 65,Västra Götalands län,Göteborg,1480,Göteborg,SE,...,0.0,6.0,2019-09-03T10:37:05.0592506+02:00,2021-07-16T11:58:06.8046101+02:00,NaN,NaN,NaN,6,Ja,Parkering Göteborg
2,2022-06-14T17-27-54,0bddfef0-0363-4e82-ba62-b428f0e6de88,STF Wendelsberg Hotell och Vandrarhem,Oskar Lundgrens Väg 3B,43535,Västra Götalands län,Härryda,1401,Mölnlycke,SE,...,11.0,2.0,2019-08-02T18:27:24.7251837+02:00,2021-02-21T09:28:00.3275998+01:00,NaN,NaN,NaN,NaN,NaN,STF Wendelsberg hotell och hostel
3,2022-06-14T17-27-54,e6e04351-4aaf-4e1d-8d6a-d4994e33b418,Signal & Andersson,Hulda Mellgrens gata 4,421 32,Västra Götalands län,Mölndal,1481,Västra Frölunda,SE,...,0.0,1.0,2019-10-18T10:00:32.4381494+02:00,2019-10-18T10:00:32.4381494+02:00,NaN,NaN,NaN,1,NaN,Signal & Andersson
4,2022-06-14T17-27-54,a1f46b96-6fa9-4cfc-b6bc-f5e0c44a9529,Torggatan 3,Torggatan 3,431 35,Västra Götalands län,Mölndal,1481,Mölndal,SE,...,3.7,4.0,2021-05-03T18:31:13.1486696+02:00,2021-05-03T18:31:13.1486696+02:00,EWAYS SE,Kundtjänst,NaN,NaN,NaN,Hemsö Juristen 4 AB


In [26]:
df = df.loc[:, ['id', 'lat', 'lng', 'pw', 'n']]
df.loc[:, 'fast'] = df.apply(lambda row: row['n'] if row['pw'] > 22 else 0, axis=1)

In [27]:
gdf = dw.df2gdf_point(df, 'lng', 'lat', crs=4326, drop=True).to_crs(3006)
gdf.head()

,id,pw,n,fast,geometry
0,a6232963-fe8d-44c2-8a33-f1b1f97f2fc8,3.7,6.0,0.0,POINT (313854.917 6426852.422)
1,ea1d02aa-fa06-400f-b141-6d5f024f9b34,0.0,6.0,0.0,POINT (321076.678 6400932.768)
2,0bddfef0-0363-4e82-ba62-b428f0e6de88,11.0,2.0,0.0,POINT (328382.852 6394960.833)
3,e6e04351-4aaf-4e1d-8d6a-d4994e33b418,0.0,1.0,0.0,POINT (318565.876 6393579.914)
4,a1f46b96-6fa9-4cfc-b6bc-f5e0c44a9529,3.7,4.0,0.0,POINT (322206.239 6394693.635)


## Load simulation results

In [36]:
df_sim = pd.read_csv(f"results/{scenario}_stats_charger_5days.csv")
df_sim = df_sim.loc[df_sim['power_fast'] == 50, :]
df_sim = df_sim.groupby(['deso', 'Charging_type'])['number'].sum().reset_index()
df_sim.head()

,deso,Charging_type,number
0,0114C1050,1 (0.2),1.0
1,0114C1050,2 (0.2),1.0
2,0114C1050,3 (0.9),1.0
3,0127C1130,1 (0.2),1.0
4,0127C1130,2 (0.2),1.0


In [35]:
zones = gpd.read_file('dbs/DeSO/DeSO_2018_v2.shp')
zones = zones.loc[zones['deso'].isin(df_sim['deso'].unique()), :]

## Calculate existing number of charging points

In [30]:
gdf = gpd.sjoin(gdf, zones.loc[:, ['deso', 'befolkning', 'geometry']])
gdf.head()

,id,pw,n,fast,geometry,index_right,deso,befolkning
0,a6232963-fe8d-44c2-8a33-f1b1f97f2fc8,3.7,6.0,0.0,POINT (313854.917 6426852.422),4194,1482B2030,1878
1,ea1d02aa-fa06-400f-b141-6d5f024f9b34,0.0,6.0,0.0,POINT (321076.678 6400932.768),4023,1480C2810,1641
2,0bddfef0-0363-4e82-ba62-b428f0e6de88,11.0,2.0,0.0,POINT (328382.852 6394960.833),3576,1401C1090,1714
1036,c6b255b0-3b17-46e7-8936-25338175cc93,22.0,6.0,0.0,POINT (327950.927 6394331.454),3576,1401C1090,1714
1043,14e4f771-ac65-4515-b896-69091925ccf9,22.0,8.0,0.0,POINT (327782.782 6394274.965),3576,1401C1090,1714


In [31]:
df_inf = gdf.groupby('deso')['n'].sum().reset_index()
df_inf.head()

,deso,n
0,1401A0020,97.0
1,1401B2010,36.0
2,1401C1010,2.0
3,1401C1040,5.0
4,1401C1060,6.0


In [32]:
df = pd.merge(df_sim, df_inf, on='deso', how='outer')
df = df.fillna(0)
df.columns = ['deso', 'Charging_type', 'sim', 'gt']
df.head()

,deso,Charging_type,sim,gt
0,0114C1050,1 (0.2),1.0,0.0
1,0114C1050,2 (0.2),1.0,0.0
2,0114C1050,3 (0.9),1.0,0.0
3,0127C1130,1 (0.2),1.0,0.0
4,0127C1130,2 (0.2),1.0,0.0


In [33]:
# Sale up to total VG residents
df.sim = df.sim * (1/0.35) # The results are based on 35% VG car users.
df.to_csv(f'results/{scenario}_inf_comp.csv', index=False)